# Relational Database for Movie Rentals Exploration
## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#mostrent">Top 3 Countries with most rentals</a></li>
<li><a href="#popfilm">Most popular film category per country</a></li>
<li><a href="#percentchange">Monthly customer count percent change per store</a></li>
<li><a href="#avgpayment">Customers' average monthly payment (usd) per store?</a></li>
<li><a href="#rentchinaindia">Top 3 movies in China and India</a></li>
</ul>

<a id='intro'></a>
## Introduction

This project will use SQL to explore a relational database related to movie rentals to gain an understanding of the customer base.

### Database Description
The database used is Sakila DVD Rental database. It is a PostgreSQL sample database from the 
[PostgreSQL tutorial](https://www.postgresqltutorial.com/postgresql-getting-started/postgresql-sample-database/).
The database represents the business processes of a DVD rental store. There are 15 tables in the database. An ER model of the database is provided below.

![DVD Rental ER Model](https://www.postgresqltutorial.com/wp-content/uploads/2018/03/dvd-rental-sample-database-diagram.png)

#### What kind of data each table holds:
- actor – stores actors data including first name and last name.
- film – stores film data such as title, release year, length, rating, etc.
- film_actor – stores the relationships between films and actors.
- category – stores film’s categories data.
- film_category- stores the relationships between films and categories.
- store – contains the store data including manager staff and address.
- inventory – stores inventory data.
- rental – stores rental data.
- payment – stores customer’s payments.
- staff – stores staff data.
- customer – stores customer data.
- address – stores address data for staff and customers
- city – stores city names.
- country – stores country names.

#### Notes
- Australia was removed from the result because there are no customer with an Australian address.
- Database contains only 2007 payments when rentail dates are all in 2005 and 2006.

<a id='mostrent'></a>
## What are the top 3 countries that rent the most movies?

In [86]:
%%capture --no-display
%%sql

SELECT country, COUNT(rental_id) as rental_count
    FROM country c
    -- Join tables to get columns needed for analysis
    JOIN city ct
        ON c.country_id = ct.country_id
    JOIN address a
        ON a.city_id = ct.city_id
    JOIN customer cust
        ON cust.address_id = a.address_id
    JOIN rental r
        ON r.customer_id = cust.customer_id
    GROUP BY country
    ORDER BY rental_count DESC
    LIMIT 3;

country,rental_count
India,1572
China,1426
United States,968


### What are the top 3 countries that rent the most each year?

In [87]:
%%capture --no-display
%%sql

-- Filter query for only top 3 countries with most rental per year
SELECT  rental_year, country, rental_count
    -- Create table that has all the country ranked by rental count each year
    FROM (SELECT country, CAST(DATE_PART('year', rental_date) AS int) AS rental_year, COUNT(rental_id) AS rental_count,
          -- Rank each country per  year reference: https://www.sqlshack.com/overview-of-sql-rank-functions/
            RANK() OVER(PARTITION BY CAST(DATE_PART('year', rental_date) AS int) ORDER BY COUNT(rental_id) DESC) rank
        FROM country c
        -- Join tables to get columns needed for analysis
        JOIN city ct
            ON c.country_id = ct.country_id
        JOIN address a
            ON a.city_id = ct.city_id
        JOIN customer cust
            ON cust.address_id = a.address_id
        JOIN rental r
            ON r.customer_id = cust.customer_id
        GROUP BY rental_year, country
        ORDER BY rental_count DESC) AS t
    WHERE rank <= 3;


rental_year,country,rental_count
2005,India,1549
2005,China,1413
2005,United States,957
2006,India,23
2006,China,13
2006,United States,11


<a id='popfilm'></a>
## What is the most popular film category in each country?

In [99]:
%%capture  --no-stderr
%%sql

/* Query output too large for ease of use in notebook so it was export to csv file*/
WITH
    -- Create subquery of each category count per country
    category_per_country AS (
        SELECT c.country, ca.name category, COUNT(*) category_count
            FROM country c
            JOIN city ct
                ON c.country_id = ct.country_id
            JOIN address a
                ON ct.city_id = a.city_id
            JOIN customer cu
                ON a.address_id = cu.address_id
            JOIN rental r
                ON cu.customer_id = r.customer_id
            JOIN inventory i
                ON r.inventory_id = i.inventory_id
            JOIN film f
                ON i.film_id = f.film_id
            JOIN film_category fc
                ON f.film_id =fc.film_id
            JOIN category ca
                ON fc.category_id = ca.category_id
            GROUP BY 1,2),
    -- Create subquery of the most rental count per country
    max_per_country AS (
        SELECT country, MAX(category_count) max_count
            FROM category_per_country
            GROUP BY 1),
    -- Create subquery of most rental count with its category in each country
    max_w_category AS (
        SELECT c.country, c.category, m.max_count
            FROM category_per_country c
            JOIN max_per_country m
                ON c.country = m.country
                AND c.category_count = m.max_count)

-- Create query of countries and their most popular movie category
SELECT country,
       CASE WHEN category = 'Action' THEN 
            max_count ELSE 0 END AS action,
       CASE WHEN category = 'Animation' THEN 
            max_count ELSE 0 END AS animation,
       CASE WHEN category = 'Children' THEN 
            max_count ELSE 0 END AS children,
       CASE WHEN category = 'Classics' THEN 
            max_count ELSE 0 END AS classics,
       CASE WHEN category = 'Comedy' THEN 
            max_count ELSE 0 END AS comedy,
       CASE WHEN category = 'Documentary' THEN 
            max_count ELSE 0 END AS documentary,
       CASE WHEN category = 'Drama' THEN 
            max_count ELSE 0 END AS drama,
       CASE WHEN category = 'Family' THEN 
            max_count ELSE 0 END AS family,
       CASE WHEN category = 'Foreign' THEN 
            max_count ELSE 0 END AS foreign,
       CASE WHEN category = 'Games' THEN 
            max_count ELSE 0 END AS games,
       CASE WHEN category = 'Horror' THEN 
            max_count ELSE 0 END AS horror,
       CASE WHEN category = 'Music' THEN 
            max_count ELSE 0 END AS music,
       CASE WHEN category = 'New' THEN 
            max_count ELSE 0 END AS new,
       CASE WHEN category = 'Sci-Fi' THEN 
            max_count ELSE 0 END AS sci_fi,
       CASE WHEN category = 'Sports' THEN 
            max_count ELSE 0 END AS sports,
       CASE WHEN category = 'Travel' THEN 
            max_count ELSE 0 END AS travel
    FROM max_w_category
    ORDER BY 2 DESC, 3 DESC, 4 DESC, 5 DESC,
       6 DESC, 7 DESC, 8 DESC, 9 DESC, 10 DESC,
       11 DESC, 12 DESC, 13 DESC, 14 DESC,
       15 DESC, 16 DESC, 17 DESC

<a id='percentchange'></a>
## What is the percent change in the number of customers renting at each store?

In [89]:
%%capture --no-display
%%sql

/*2006 data was removed from result in case it was incomplete.*/
WITH 
    -- Create subquery of the customers renting movies at each store per month
    customer_per_month AS (
        SELECT DISTINCT st.store_id, DATE_TRUNC('month',r.rental_date) rental_month, r.customer_id
            FROM rental r
            JOIN payment p
                ON r.rental_id = p.rental_id
            JOIN staff s
                ON p.staff_id = s.staff_id
            JOIN store st
                ON s.store_id = st.store_id),
    -- Create subquery of customer count at each store per month
    month_count AS (
        SELECT store_id, rental_month, COUNT(*) customer_count
            FROM customer_per_month
            GROUP BY 1,2)

/* Append query of customer percent change for store 2 to store 1 query.*/
-- Create query calculating customer percent change for each month in store 1
SELECT store_id,
    DATE_PART('year',rental_month)||'-0'||DATE_PART('month',rental_month) AS year_month,
    CASE WHEN (customer_count - LAG(customer_count) OVER(ORDER BY rental_month)) IS NULL
        THEN 0
        ELSE 100*(customer_count - LAG(customer_count) OVER(ORDER BY rental_month))/customer_count 
        END AS customer_percent_change
    FROM month_count
    WHERE store_id = 1
        AND (rental_month BETWEEN '2005-06-01' AND '2006-01-01')
UNION ALL
-- Create query calculating customer percent change for each month in store 2
SELECT store_id, 
    DATE_PART('year',rental_month)||'-0'||DATE_PART('month',rental_month) AS year_month,
    CASE WHEN (customer_count - LAG(customer_count) OVER (ORDER BY rental_month)) IS NULL
        THEN 0
        ELSE 100*(customer_count - LAG(customer_count) OVER(ORDER BY rental_month))/customer_count 
        END AS customer_percent_change
    FROM month_count
    WHERE store_id = 2
        AND (rental_month BETWEEN '2005-06-01' AND '2006-01-01')
    ORDER BY 1,2;

store_id,year_month,customer_percent_change
1,2005-06,0
1,2005-07,24
1,2005-08,-1
2,2005-06,0
2,2005-07,23
2,2005-08,0


<a id='avgpayment'></a>
## What is the average amount (usd) customers pay per month at each store?

In [90]:
%%capture --no-display
%%sql

-- Create subquery calculating the average payment per month for each store
WITH avg_table AS (
    SELECT st.store_id, DATE_TRUNC('month',p.payment_date) payment_month, ROUND(AVG(p.amount),2) avg_payment
        FROM payment p
        JOIN staff s
            ON p.staff_id = s.staff_id
        JOIN store st
            ON s.store_id = st.store_id
        GROUP BY 1,2)

-- Create query to sort average payment by store and month
SELECT store_id, DATE_PART('year',payment_month)||'-0'||DATE_PART('month',payment_month) year_month, avg_payment
    FROM avg_table
    ORDER BY 1,2;

store_id,year_month,avg_payment
1,2007-02,4.10
1,2007-03,4.18
1,2007-04,4.19
1,2007-05,2.46
2,2007-02,4.19
2,2007-03,4.28
2,2007-04,4.27
2,2007-05,3.22


<a id='rentchinaindia'></a>
## What are the top 3 most rented movies in China and India?

In [92]:
%%capture --no-display
%%sql

-- Create subquery ranking movies based on rental count for China and India
WITH movie_count AS (
    SELECT c.country, f.title, COUNT(*) rent_count, f.rental_rate, 
    RANK() OVER (PARTITION BY country ORDER BY COUNT(*) DESC) movie_rank
        FROM country c
        JOIN city ct
            ON c.country_id = ct.country_id
        JOIN address a
            ON ct.city_id = a.city_id
        JOIN customer cu
            ON a.address_id = cu.address_id
        JOIN rental r
            ON cu.customer_id = r.customer_id
        JOIN inventory i
            ON r.inventory_id = i.inventory_id
        JOIN film f
            ON i.film_id = f.film_id
        WHERE c.country = 'China'
            OR c.country = 'India'
        GROUP BY 1,2,4)
 
-- Create query for top 3 most rented movies in China and India
 SELECT *
    FROM movie_count
    WHERE movie_rank <= 3;

country,title,rent_count,rental_rate,movie_rank
China,Story Side,8,0.99,1
China,Balloon Homeward,7,2.99,2
China,Packer Madigan,7,0.99,2
India,Wife Turn,9,4.99,1
India,Innocent Usual,8,4.99,2
India,Secret Groundhog,7,4.99,3
India,Sabrina Midnight,7,4.99,3
India,Fiction Christmas,7,0.99,3
